<a href="https://colab.research.google.com/github/SaziaSDark/ETL_Coursera/blob/main/ETL_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url='https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
page=requests.get(url).text
soup=BeautifulSoup(page,'html.parser')

table=soup.find_all('tbody')
#rows=table[2].find_all('tr')
#p3rint(rows)
titles=soup.find_all('th')[:3:2]
column_titles = [header.text.strip() for header in soup.find_all('th')]
titles2=[title.text.strip() for title in titles]
#print(titles2)

df=pd.DataFrame(columns=titles2)

column_data=soup.find_all("tr")[5:219]
for row in column_data[1:]:
  row_data=row.find_all("td")[:3:2]
  individual_row_data=[data.text.strip() for data in row_data]
  length=len(df)
  df.loc[length]=individual_row_data
print(df)
#rows=table[2].find_all('tr')
#print(rows)

ConnectTimeout: ignored

In [ ]:
import pandas as pd
data={'bank name': ['sazia bank',"shahreen bank"],"market cap$":[2000,4000],}
df4=pd.DataFrame(data)
print(df4)

       bank name  market cap$
0     sazia bank         2000
1  shahreen bank         4000


In [50]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
def ex():
  url='https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'
  page=requests.get(url).text
  soup=BeautifulSoup(page,'html.parser')

  table=soup.find_all('table')
  selected_table = table[0]
  titles=selected_table.find_all('th')[1:]


  columns=[data.text.strip() for data in titles]
  print(columns)
  df=pd.DataFrame(columns=columns)
  for row in selected_table.find_all('tr')[1:]:
    row_data=[data.text.strip() for data in row.find_all('td')[1:]]
    print(row_data)
    length=len(df)
    df.loc[length]=row_data
  return df
ex()
#load df to sql table


['Bank name', 'Market cap(US$ billion)']
['JPMorgan Chase', '432.92']
['Bank of America', '231.52']
['Industrial and Commercial Bank of China', '194.56']
['Agricultural Bank of China', '160.68']
['HDFC Bank', '157.91']
['Wells Fargo', '155.87']
['HSBC Holdings PLC', '148.90']
['Morgan Stanley', '140.83']
['China Construction Bank', '139.82']
['Bank of China', '136.81']


,Bank name,Market cap(US$ billion)
0,JPMorgan Chase,432.92
1,Bank of America,231.52
2,Industrial and Commercial Bank of China,194.56
3,Agricultural Bank of China,160.68
4,HDFC Bank,157.91
5,Wells Fargo,155.87
6,HSBC Holdings PLC,148.90
7,Morgan Stanley,140.83
8,China Construction Bank,139.82
9,Bank of China,136.81


In [48]:
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''

    # Fetching the webpage content
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'html.parser')

    # Extracting table
    table = soup.find_all('table')
    selected_table = table[0]

    # Extracting column titles
    titles = selected_table.find_all('th')[1:]
    columns = [data.text.strip() for data in titles]

    # Creating an empty DataFrame with columns
    df = pd.DataFrame(columns=columns)

    # Extracting rows and populating DataFrame
    for row in selected_table.find_all('tr')[1:]:
        row_data = [data.text.strip() for data in row.find_all('td')[1:]]
        length = len(df)
        df.loc[length] = row_data

    return df

In [ ]:

df2=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")
print(df2.head())
df["Market cap(EUR billion)"]=(df["Market cap(US$ billion)"].astype(float)*df2["Rate"].loc[0].astype(float)).round(2)
df["Market cap(GBP billion)"]=(df["Market cap(US$ billion)"].astype(float)*df2["Rate"].loc[1].astype(float)).round(2)
df["Market cap(INR billion)"]=(df["Market cap(US$ billion)"].astype(float)*df2["Rate"].loc[2].astype(float)).round(2)
print(df)



  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95
                                 Bank name Market cap(US$ billion)  \
0                           JPMorgan Chase                  432.92   
1                          Bank of America                  231.52   
2  Industrial and Commercial Bank of China                  194.56   
3               Agricultural Bank of China                  160.68   
4                                HDFC Bank                  157.91   
5                              Wells Fargo                  155.87   
6                        HSBC Holdings PLC                  148.90   
7                           Morgan Stanley                  140.83   
8                  China Construction Bank                  139.82   
9                            Bank of China                  136.81   

   Market cap(EUR billion)  Market cap(GBP billion)  Market cap(INR billion)  
0                   402.62                   346.34                 35910.71  

In [ ]:

import sqlite3
db_path="bank_data.db"
conn=sqlite3.connect(db_path)
table_name="bank_marketcap"
df.to_sql(table_name,conn,index=False,if_exists='replace')
cursor=conn.cursor()
query='SELECT "Bank name","Market cap(GBP billion)" FROM bank_marketcap '
cursor.execute(query)
result=cursor.fetchall()
for row,name in result:
  print(row,name)

conn.close()
print(f"DataFrame successfully loaded into SQLite table '{table_name}'.")


JPMorgan Chase 346.34
Bank of America 185.22
Industrial and Commercial Bank of China 155.65
Agricultural Bank of China 128.54
HDFC Bank 126.33
Wells Fargo 124.7
HSBC Holdings PLC 119.12
Morgan Stanley 112.66
China Construction Bank 111.86
Bank of China 109.45
DataFrame successfully loaded into SQLite table 'bank_marketcap'.


In [ ]:
#alternative:
def extract(url, table_attribs):
    page = requests.get(url).text
    data = BeautifulSoup(page,'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[2].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            if col[0].find('a') is not None and '—' not in col[2]:
                data_dict = {"Country": col[0].a.contents[0],
                             "GDP_USD_millions": col[2].contents[0]}
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df,df1], ignore_index=True)
    return df


In [ ]:
url = 'https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
table_attribs = ["Country", "GDP_USD_millions"]
db_name = 'World_Economies.db'
table_name = 'Countries_by_GDP'
csv_path = './Countries_by_GDP.csv'

In [ ]:
# Code for ETL operations on Country-GDP data
import bs4 as BeautifulSoup
import pandas as pd
import requests

def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''

def extract(url, columns2):
  page=requests.get(url).text
  soup=BeautifulSoup(page,'html.parser')
  df=pd.DataFrame(columns=columns2)
  tables=soup.find_all('tbody')
  rows=tables[0].find_all('tr')
  df=pd.DataFrame()
  for row in rows:
    if row.find('td') is not None:
      col=row.find_all('td')
      bank_name=col[1].find_all('a')[1]['title']
      market_cap=col[2].contents[0][:-1]
      data={"Name":bank_name,"MC_USD_Billion":float(market_cap)}
      df1=pd.DataFrame(data,index=[0])
      df=pd.concat([df,df1],ignore_index=True)
  return(df)

def transform(df, csv_path):

    df2=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")
    print(df2.head())
    df["Market cap(EUR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[0].astype(float)).round(2)
    df["Market cap(GBP billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[1].astype(float)).round(2)
    df["Market cap(INR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[2].astype(float)).round(2)


    return df

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''

def run_query(query_statement, sql_connection):
    cursor=conn.cursor()
    query='SELECT "Bank name","Market cap(GBP billion)" FROM bank_marketcap '
    cursor.execute(query)
    result=cursor.fetchall()
    for row,name in result:
      print(row,name)

''' Here, you define the required entities and call the relevant
functions in the correct order to complete the project. Note that this
portion is not inside any function.'''

In [108]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
def extract(url, columns2):
  page=requests.get(url).text
  soup=BeautifulSoup(page,'html.parser')
  df=pd.DataFrame(columns=columns2)
  tables=soup.find_all('tbody')
  rows=tables[0].find_all('tr')
  df=pd.DataFrame()
  for row in rows:
    if row.find('td') is not None:
      col=row.find_all('td')
      bank_name=col[1].find_all('a')[1]['title']
      market_cap=col[2].contents[0][:-1]
      data={"Name":bank_name,"MC_USD_Billion":float(market_cap)}
      df1=pd.DataFrame(data,index=[0])
      df=pd.concat([df,df1],ignore_index=True)
  return(df)
def transform(df, csv_path):
    df2=pd.read_csv(csv_path)
    df["MC_EUR_billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[0].astype(float)).round(2)
    df["MC_GBP_billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[1].astype(float)).round(2)
    df["MC_INR_billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[2].astype(float)).round(2)


    return df



url="https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"
page=requests.get(url).text
soup=BeautifulSoup(page,'html.parser')

table=soup.find_all('tbody')
selected_table = table[0]
titles=selected_table.find_all('th')[1:]
titles1=soup.find_all('tbody')[0].find_all('th')[1:]
columns2=[data.text.strip() for data in titles]
extract(url,columns)

extracted_df=extract(url,columns)
csv="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
transform(extracted_df,csv)

conn.close()

In [110]:
csv_new='exchange_rate.csv'
load_to_csv(extracted_df,csv)

import sqlite3
db_path="bank_data.db"
conn=sqlite3.connect(db_path)
table_name="bank_marketcap"

load_to_db(extracted_df, conn, table_name)

print(f"DataFrame successfully loaded into SQLite table '{table_name}'.")

query2='SELECT AVG(MC_GBP_billion) FROM bank_marketcap'

run_query(query2,conn)

DataFrame successfully loaded into SQLite table 'bank_marketcap'.


OperationalError: no such column: MC_GBP_billion

In [121]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
def extract(url, columns2):
  page=requests.get(url).text
  soup=BeautifulSoup(page,'html.parser')
  df=pd.DataFrame(columns=columns2)
  tables=soup.find_all('tbody')
  rows=tables[0].find_all('tr')
  df=pd.DataFrame()
  for row in rows:
    if row.find('td') is not None:
      col=row.find_all('td')
      bank_name=col[1].find_all('a')[1]['title']
      market_cap=col[2].contents[0][:-1]
      data={"Name":bank_name,"MC_USD_Billion":float(market_cap)}
      df1=pd.DataFrame(data,index=[0])
      df=pd.concat([df,df1],ignore_index=True)
  return(df)
def transform(df, csv_path):
    df2=pd.read_csv(csv_path)
    df["Market cap(EUR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[0].astype(float)).round(2)
    df["MC_GBP_billion"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[1].astype(float)).round(2)
    df["Market cap(INR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[2].astype(float)).round(2)


    return df
def load_to_csv(df, output_path):
  csv_new=df.to_csv(output_path,index=False)



def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, conn):
    cursor=conn.cursor()
    cursor.execute(query_statement)
    result=cursor.fetchall()
    output=[]
    print(query_statement)
    for row in result:
      print(row)
url="https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks"
page=requests.get(url).text
soup=BeautifulSoup(page,'html.parser')

table=soup.find_all('tbody')
selected_table = table[0]
titles=selected_table.find_all('th')[1:]
titles1=soup.find_all('tbody')[0].find_all('th')[1:]
columns2=[data.text.strip() for data in titles]
extract(url,columns)

extracted_df=extract(url,columns)
csv="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
transform(extracted_df,csv)
csv_new='exchange_rate.csv'
load_to_csv(extracted_df,csv)

import sqlite3
db_path="bank_data.db"
conn=sqlite3.connect(db_path)
table_name="bank_marketcap"

load_to_db(extracted_df, conn, table_name)

print(f"DataFrame successfully loaded into SQLite table '{table_name}'.")

query2='SELECT AVG(MC_GBP_billion) FROM bank_marketcap'
query3='SELECT Name FROM bank_marketcap LIMIT 5'
run_query(query,conn)
run_query(query2,conn)
run_query(query3,conn)


DataFrame successfully loaded into SQLite table 'bank_marketcap'.
SELECT * FROM bank_marketcap 
('JPMorgan Chase', 432.92, 402.62, 346.34, 35910.71)
('Bank of America', 231.52, 215.31, 185.22, 19204.58)
('Industrial and Commercial Bank of China', 194.56, 180.94, 155.65, 16138.75)
('Agricultural Bank of China', 160.68, 149.43, 128.54, 13328.41)
('HDFC Bank', 157.91, 146.86, 126.33, 13098.63)
('Wells Fargo', 155.87, 144.96, 124.7, 12929.42)
('HSBC', 148.9, 138.48, 119.12, 12351.26)
('Morgan Stanley', 140.83, 130.97, 112.66, 11681.85)
('China Construction Bank', 139.82, 130.03, 111.86, 11598.07)
('Bank of China', 136.81, 127.23, 109.45, 11348.39)
SELECT AVG(MC_GBP_billion) FROM bank_marketcap
(151.98700000000002,)
SELECT Name FROM bank_marketcap LIMIT 5
('JPMorgan Chase',)
('Bank of America',)
('Industrial and Commercial Bank of China',)
('Agricultural Bank of China',)
('HDFC Bank',)


In [ ]:
df.to_sql(table_name,conn,index=False,if_exists='replace')
cursor=conn.cursor()
query='SELECT "Bank name","Market cap(GBP billion)" FROM bank_marketcap '
cursor.execute(query)
result=cursor.fetchall()
for row,name in result:
  print(row,name)

In [112]:
def transform(df, csv_path):
    df2=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv")
    print(df2.head())
    df["Market cap(EUR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[0].astype(float)).round(2)
    df["Market cap(GBP billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[1].astype(float)).round(2)
    df["Market cap(INR billion)"]=(df["MC_USD_Billion"].astype(float)*df2["Rate"].loc[2].astype(float)).round(2)


    return df
    csv="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv"
transform(extracted_df,csv)

  Currency   Rate
0      EUR   0.93
1      GBP   0.80
2      INR  82.95


,Name,MC_USD_Billion,Market cap(EUR billion),MC_GBP_billion),Market cap(INR billion),Market cap(GBP billion)
0,JPMorgan Chase,432.92,402.62,346.34,35910.71,346.34
1,Bank of America,231.52,215.31,185.22,19204.58,185.22
2,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75,155.65
3,Agricultural Bank of China,160.68,149.43,128.54,13328.41,128.54
4,HDFC Bank,157.91,146.86,126.33,13098.63,126.33
5,Wells Fargo,155.87,144.96,124.70,12929.42,124.70
6,HSBC,148.90,138.48,119.12,12351.26,119.12
7,Morgan Stanley,140.83,130.97,112.66,11681.85,112.66
8,China Construction Bank,139.82,130.03,111.86,11598.07,111.86
9,Bank of China,136.81,127.23,109.45,11348.39,109.45


In [ ]:
def load_to_csv(df, output_path):
  csv_new=df.to_csv(output_path,index=False)



def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)

def run_query(query_statement, conn):
    cursor=conn.cursor()
    cursor.execute(query_statement)
    result=cursor.fetchall()
    print(query_statement)
    for row in result:
      print(row)